In [1]:
import json_lines as jls
import re
import matplotlib
import numpy as np
from jedi.api.refactoring import inline
from tqdm import tqdm
import matplotlib.pyplot as plt
np.random.seed(42)

In [2]:
TRAIN = " "
if TRAIN == "test":
    train_file = 'contents_test.jl'
    weight_dir = 'Weights_fnn_small'
else:
    train_file = 'contents_small.jl'
    weight_dir = 'Weights_fnn'
print("train_file: ", train_file)
print("weight_dir: ", weight_dir)

train_file:  contents_small.jl
weight_dir:  Weights_fnn


In [3]:
word_to_id = {}
word_to_id = np.load(f'{weight_dir}/dict_fnn.npy',allow_pickle=True).item()
id_to_word = {}
for key in word_to_id:
    id_to_word[word_to_id[key]] = key
print(len(word_to_id))

1001


In [4]:
embedding_num = 10
model = {}
model["w1"] = np.load(f'{weight_dir}/weight_1_20.npy')
model["w2"] = np.load(f'{weight_dir}/weight_2_20.npy')
def get_range(*iterables):
    for iterable in iterables:
        yield from iterable

def one_hot_encode(id, vocab_size, s=False):
    res = [0] * vocab_size

    res[id] = 1
    if s==True:
        print(len(res))
    return res
def generate_test_data(tokens, word_to_id):
    X = []
    for i in range(0, 4):
        X.append(one_hot_encode(word_to_id[tokens[i].lower()],
                                len(word_to_id)))
    return np.asarray(X)
def softmax(X):
    res = []
    for x in X:
        exp = np.exp(x - np.max(x))
        res.append(exp / exp.sum())
    return res
def cross_entropy(z, y):
    for i, array in enumerate(z):
        z[i] = array + len(array)*[0.0001]
    return - np.sum(np.log(z) * y)
def get_aver_matrix(k1):
    AVER = np.array([[0]*k1*4]*k1)
    TEMP = []
    for i, row in enumerate(AVER):
        # print(i)
        child_TEMP = list(row)
        child_TEMP[4*i] = 0.25
        child_TEMP[4*i+1] = 0.25
        child_TEMP[4*i+2] = 0.25
        child_TEMP[4*i+3] = 0.25
        TEMP.append(np.array(child_TEMP))
    AVER = np.array(TEMP)
    return AVER
def forward(model, X, return_cache=True):
    cache = {}
    cache["a1"] = X @ model["w1"]
    k1 = int(X.shape[0]/4)
    AVER = get_aver_matrix(k1)
    cache["b"] = AVER @ cache["a1"]
    cache["a2"] = cache["b"] @ model["w2"]
    cache["z"] = softmax(cache["a2"])
    if not return_cache:
        print("W1.shape: ",model["w1"].shape)
        print("A1.shape: ",cache["a1"].shape)
        print("B.shape: ",cache["b"].shape)
        print("A2.shape: ",cache["a2"].shape)
        print(type(cache["z"]))
        return cache["z"]
    return cache
def generate_test_data(tokens, word_to_id):
    X = []
    for i in range(0, 4):
        X.append(one_hot_encode(word_to_id[tokens[i].lower()],
                                len(word_to_id)))
    return np.asarray(X)

In [5]:
learning = generate_test_data(["rich","cultural","and","natural"], word_to_id)
learn = np.array(learning)
print("learn shape", learn.shape)
result = forward(model, learn, return_cache=False)[0]
print("result shape", result.__len__())
id = 0
temp = 0
for i, prob in enumerate(result):
    if temp == 0:
        temp = prob
        id = i
    elif prob > temp:
        temp = prob
        id = i
print("most possible: ",id,result[id])
print(id_to_word[id])

learn shape (4, 1001)
W1.shape:  (1001, 10)
A1.shape:  (4, 10)
B.shape:  (1, 10)
A2.shape:  (1, 1001)
<class 'list'>
result shape 1001
most possible:  600 0.024744032272691258
doses


In [6]:
Vectors = np.load(f'{weight_dir}/weight_1_20.npy')
print(Vectors.shape)
def ed(m, n):
 return np.sqrt(np.sum((m - n) ** 2))

(1001, 10)


In [7]:
for i in range(20):
    words = []
    ID = int(np.random.uniform(0,169))
    # print("WORD: ",id_to_word[ID])
    words.append(id_to_word[ID])
    VEC = Vectors[ID]
    dist_to_id = {}
    dists = []
    for i, vector in enumerate(Vectors):
        dist = ed(vector,VEC)
        dist_to_id[dist]=i
        dists.append(dist)
    dists.sort(reverse=True)
    for i, dist in enumerate(dists):
        if i == 10:
            break
        words.append(id_to_word[dist_to_id[dist]])
    print(words)

['chew', 'inflation', 'heritage', 'ethics', 'reinforced', 'adults', 'together', 'four', 'continued', 'relations', 'household']
['rich', 'means', 'inflation', 'under', 'indicating', 'event', 'continued', 'adults', 'upali', 'risk', 'away']
['become', 'seen', 'beverages', 'grains', 'region', 'rhythmic', 'attended', 'promoting', 'aratchilage', 'sure', 'reduction']
['southern', 'movement', 'due', 'thursday', 'resources', 'rose', 'indicating', 'matshidiso', 'aratchilage', 'after', 'within']
['s', 'inflation', 'need', 'seen', 'beverages', 'benefit', 'e', 'continued', 'into', 'drought', 'reinforced']
['s', 'inflation', 'need', 'seen', 'beverages', 'benefit', 'e', 'continued', 'into', 'drought', 'reinforced']
['express', 'inflation', 'need', 'away', 'ethics', 'reinforced', 'e', 'adults', 'heritage', 'become', 'after']
['expressed', 'into', 'four', 'forward', 'parents', 'beverages', 'sure', 'benefit', 'risk', 'seen', 'had']
['southern', 'movement', 'due', 'thursday', 'resources', 'rose', 'indica